In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
import pandas as pd
import bs4
from langdetect import detect
import pandas as pd

In [1]:
class Scraping:
    """
    cette classe va nous permettre de récupérer le contenu de nos pages webs
    """
    
    
    
    def web_name(self, url):
        """
        this function allows to obtain the name which is in url, for example for this url https://www.specshop.pl, the name will be specshop

        Args:
            url (url): url for the web page

        Returns:
            str: the name that located of the url
        """
        
        return "".join(urlparse(url).netloc.split(".")[-2])
    
    def web_title(self, soup):
        """

        Args:
            soup (BeautifulSoup): get the whole html parse document of the web page 

        Returns:
            str: title of the page
        """
        return " ".join(soup.title.contents) if soup.title is not None else ""

    
    def web_language(self, soup):
        """

        Args:
            soup (BeautifulSoup): get the whole html parse document of the web page 

        Returns:
            str: language of the page
        """
        try:
            language = detect(soup.get_text())
        except Exception:
            language = "en"

        return language

 
 
    def web_meta(self, soup):
        """
        gets some meta data from the web page header
        
        Args:
            soup (BeautifulSoup): get the whole html parse document of the web page

        Returns:
            str: the description of our page in the header
        """
        
        tags = soup.find_all(lambda tag: (tag.name=='meta') & (tag.has_attr('name') & tag.has_attr('content')))
        
        content = [str(tag['content']) for tag in tags if tag['name'] in ['keywords', 'description']]
        return " ".join(content)
    
    
    def web_header(self, soup):
        """

        Args:
            soup (BeautifulSoup): get the whole html parse document of the web page

        Returns:
            str: all titles from h1 to h6 of the web page
        """
        
        tags = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])
        if not tags:
            return ""
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return " ".join(content)
    
    def web_contents(self, soup):
        """
        
        Args:
            soup (BeautifulSoup): get the whole html parse document of the web page


        Returns:
            str: any other element of our html that is not a title, css style, etc...
        """
        
        tags_to_ignore = ["h1", "h2", "h3", "h4", "h5","h6", "noscript", "style", "script", "head", "title", "meta", "[document]"]
        contends = soup.find_all(text=True)
        result = []
        for word in contends:
            stripped_word = word.strip()
            if (
                word.parent.name in tags_to_ignore
                or isinstance(word, bs4.element.Comment)
                or stripped_word.isnumeric()
                or len(stripped_word) <= 0
            ):
                return ""
            result.append(stripped_word)
        return " ".join(result)
        
        
    def url_contents(self, url):
        """
        return de content of web page
        
        Args:
            url (str): url for the web page

        Returns:
            Series: content of the web page
        """
        
        
        try:
            headers = { "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/111.0"}

            response = requests.get(url, timeout=60, headers=headers)

            if response.status_code != 200:
                result = {
                    "lang": "None",
                    "url": "None",
                    "website_name": "None",
                    "content_text": "None"
                }
            else:
                soup = BeautifulSoup(response.content, "html.parser")


                result = {
                    "lang": self.get_language(soup),
                    "url": url,
                    "website_name": self.get_name(url),
                    "content_text": self.web_title(soup)+ self.web_meta(soup) + self.web_header(soup) + self.web_contents(soup)
                }

            return pd.Series(result)
        except requests.exceptions.RequestException:
            result = {
                "lang": "None",
                "url": url,
                "website_name": "None",
                "content_text": "None"
            }
            return pd.Series(result) 
        

In [3]:
import spacy

class TextCleaner:
    
    
    def __init__(self):
        self.nlp_dict = {}
        
    def clean_text(self, document, lang):
        """
        Cleans and normalizes a text according to the given language

        Args:
            document (str): a text to clean
            lang (str): the language of the document

        Returns:
            str: the normalized text
        """
        
        # load the appropriate template if you have not already done it
        if lang not in self.nlp_dict:
            lang_dict = {
                'en': 'en_core_web_sm',
                'fr': 'fr_core_news_sm',
                'de': 'de_core_news_sm',
                # add other languages if necessary
            }

            self.nlp_dict[lang] = spacy.load(lang_dict.get(lang, 'en_core_web_sm'))

        # normalizes the text with the loaded template
        doc = self.nlp_dict[lang](document)
        tokens = []
        exclusion_list = ["nan"]

        for token in doc:
            if token.is_stop or token.is_punct or token.text.isnumeric() or (token.text.isalnum() == False) or token.text in exclusion_list:
                continue

            # Normalization of the token to lowercase lemmas
            token = str(token.lemma_.lower().strip())
            tokens.append(token)
        return " ".join(tokens)
    
 



In [5]:
# test
import concurrent.futures
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor  # module for running functions in parallel
from tqdm import tqdm  # module to create a progress bar



def extract_data(domaine, scrap, cleaner):
    """
    (upgrade)Function to extract data from a line in our json file

    Args:
        domaine (dict): Line of the json file
        scrap (Scraping): instance of the Scraping class

    Returns:
        dict: dictionary containing the extracted information
    """
    
    try:
        dict_domaine = json.loads(domaine)
        name = dict_domaine['name']
        category = dict_domaine['category']
        address = dict_domaine['address']
        
        content = dict(scrap.url_contents(address))
        if content["lang"] != "None":
            content['content_text'] = content['content_text'].strip()
            if len(content['content_text'])>=20:
                text = cleaner.clean_text(content['content_text'], content['lang'])  # clean up the extracted text
                # return a dictionary containing the extracted information
                return {'name': name, 'category': category, 'address': address, 'words': text}
    except Exception as e:
        print(f"Error when extracting data for the domain {address}: {e}")
        return None
    
    
def get_dataSet(file_name: str):
    """
    (upgrade)Read a JSON file containing domain names, extract the keywords associated with each address and return a pandas Dataframe containing the extracted information

    Args:
        file_name (str): json file 

    Returns:
        Dataframe: Panda Dataframe with extracted data
    """
    
    # create an instance of the Scraping class and TextCleaner to extract data from each domain
    scrap = Scraping()
    cleaner = TextCleaner()
    
    # open the JSON file and extract the information
    with open(file_name, "r") as f:
        domaines = f.readlines()

    with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:  # execute functions in parallel with 60 threads
        futures = [executor.submit(extract_data, domaine, scrap, cleaner) for domaine in domaines]  # submit tasks
        results = []  # initialise the list to store the results
        
        with tqdm(total=len(domaines)) as pbar:  # create a progress bar to display progress
            for future in concurrent.futures.as_completed(futures):
                try:
                    result = future.result()
                    if result is not None:  # if the task has been executed successfully, add the result to the list
                        results.append(result)
                except Exception as e:
                    print(f"Error when extracting data: {e}")
                pbar.update(1)  # update of the progress bar
                
    # Create a pandas Dataframe containing the extracted data
    data = pd.DataFrame(results)
    data['indice'] = range(len(data))
    data = data.set_index('indice')
    return data

if __name__ == '__main__':
    df = get_dataSet('urls.txt') 

  0%|          | 45/119321 [01:16<29:24:49,  1.13it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  1%|          | 1307/119321 [05:33<15:29:54,  2.12it/s]/home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  2%|▏         | 1975/119321 [07:24<11:11:02,  2.91it/s]/home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
  3%|▎         | 3198/119321 [10:51<5:05:34,  6.33it/s] /home/ljatsa/.local/lib/python3.10/site-packa

Error when extracting data for the domain https://netian.com: sequence item 1: expected str instance, Tag found


  7%|▋         | 7777/119321 [24:28<5:57:27,  5.20it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 10%|▉         | 11764/119321 [37:15<9:56:09,  3.01it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 10%|█         | 12004/119321 [37:45<5:13:46,  5.70it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:43

Error when extracting data for the domain https://perfectiondistributing.com: sequence item 1: expected str instance, Tag found


 23%|██▎       | 27336/119321 [1:25:40<9:17:22,  2.75it/s] 

...

 44%|████▍     | 52208/119321 [2:36:35<5:29:53,  3.39it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 54%|█████▍    | 64172/119321 [3:10:24<5:56:32,  2.58it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 55%|█████▌    | 66194/119321 [3:16:44<6:18:56,  2.34it/s] 

Error when extracting data for the domain https://cut-the-knot.org: sequence item 1: expected str instance, Tag found


 56%|█████▋    | 67405/119321 [3:20:18<3:20:52,  4.31it/s] /home/ljatsa/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 60%|██████    | 71650/119321 [3:33:42<11:33:31,  1.15it/s]/home/ljatsa/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document

Error when extracting data for the domain https://mitsuwa-i.com: sequence item 1: expected str instance, Tag found


 81%|████████  | 96364/119321 [4:47:51<1:24:56,  4.50it/s]/home/ljatsa/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
 82%|████████▏ | 98315/119321 [4:53:30<41:17,  8.48it/s]  /home/ljatsa/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 83%|████████▎ | 98679/119321 [4:54:33<1:04:29,  5.33it/s]/home/ljatsa/.local/lib/python3.10/site-packages/bs4/builder/__

In [9]:
df["category"].unique()

array(['Illegal Activities', 'Warez / Software Piracy',
       'Computer Crime / Hacking',
       'Computer Crime / Hacking, Illegal Activities, Warez / Software Piracy',
       'Political Extreme / Hate / Discrimination',
       'Illegal Activities, Warez / Software Piracy',
       'Computer Crime / Hacking, Warez / Software Piracy',
       'Illegal Activities, Political Extreme / Hate / Discrimination',
       'Warez / Software Piracy, Computer Crime / Hacking, Illegal Activities',
       'Political Extreme / Hate / Discrimination, Illegal Activities',
       'Warez / Software Piracy, Illegal Activities',
       'Warez / Software Piracy, Computer Crime / Hacking',
       'Computer Crime / Hacking, Warez / Software Piracy, Illegal Activities',
       'Warez / Software Piracy, Political Extreme / Hate / Discrimination',
       'Alcohol', 'Tobacco', 'Self-Help / Addiction',
       'Illegal Drugs, Tobacco', 'Illegal Drugs',
       'Music / Radio Broadcast', 'Humour / Cartoons',
       'C

In [12]:

df.to_csv("mon_fichier_csv.csv", index=False)

In [ ]:
df = pd.read_csv()

In [ ]:
#first without threads
import json
import pandas as pd
from tqdm import tqdm
from typing import List

def get_dataSet(file_name: str):
    """
    Read a JSON file containing domain names, extract the keywords associated with each address and return a pandas Dataframe containing the extracted information

    Args:
        file_name (str): json file 

    Returns:
        Dataframe: Panda Dataframe with extracted data
    """
       
    # creates an instance of the Scraping class   
    scrap = Scraping()

    # open the JSON file and extract the information
    with open(file_name, "r") as f:
        domaines = f.readlines()

    # Initialise lists to store data
    indices = []
    names = []
    categories = []
    addresses = []
    keywords = []

    # Iterate through each domain name, extract the keywords associated with the address and store the data in the lists
    for index, domaine in tqdm(enumerate(domaines[:50]), total=len(domaines)):
        try:
            dict_domaine = json.loads(domaine)
            name = dict_domaine['name'] 
            category = dict_domaine['category']
            address = dict_domaine['address']
            content = dict(scrap.url_content(address))
            
            if content["lang"] != "None":
                text = clean_text(content['content_text'], content['lang'])
                keywords.append(text)

            indices.append(index)
            names.append(name)
            categories.append(category)
            addresses.append(address)
        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour le domaine à l'index {index}: {e}")

    # Create a pandas Dataframe containing the extracted data
    data = {'name': names, 'category': categories, 'address': addresses, 'words': keywords}
    return pd.DataFrame(data)


df = get_dataSet('urls.txt')



In [40]:
scrap = Scraping()
url = "https://cut-the-knot.org/"

content = dict(scrap.url_content(url))

#print(content)
print()

if content["lang"] != "None":
    text = clean_text(content['content_text'], content['lang'])
    print(text)

